# Model Evaluation: Regression & Classification

This notebook demonstrates **model evaluation** on a dataset of Swiss train stations, covering two scenarios:
1. **Regression** (predicting a numeric target, e.g., `DTV`)
2. **Classification** (predicting a binary label, e.g., High vs. Low traffic)

We will use **scikit-learn** metrics:
- **Regression**: R², MAE, RMSE
- **Classification**: Accuracy, Precision, Recall, F1, Confusion Matrix

## Contents
1. **Setup** (imports)
2. **Data Loading & Preparation**
3. **Regression** Example
4. **Classification** Example
5. **Summary & Next Steps**


In [1]:
# 1) Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix
)

# Optional: for inline plotting in Jupyter
%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

## 2. Data Loading & Preparation
We assume you have a CSV named `stations.csv` containing columns like:
- `DTV`, `DWV`, `DNWV` (numeric)
- `Kanton`, `EVU` (categorical)
- `lon`, `lat` (numeric)
… and more.

### 2.1 Load Data & Basic Cleaning

In [ ]:
# Load your CSV file
df = pd.read_csv("stations.csv")  # change if needed
print("Data shape:", df.shape)
df.head()

### 2.2 Drop Missing or Invalid Rows
We'll keep only rows where `DTV`, `DWV`, `DNWV`, `Kanton`, `EVU`, `lon`, `lat` are valid.

In [ ]:
required_cols = ['DTV','DWV','DNWV','Kanton','EVU','lon','lat']
df.dropna(subset=required_cols, inplace=True)

# Convert numeric columns
num_cols = ['DTV','DWV','DNWV','lon','lat']
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

df.dropna(subset=num_cols, inplace=True)
print("Data shape after cleaning:", df.shape)

### 2.3 Encode Categorical Features
We do one-hot encoding for `Kanton` and `EVU` so we can use them in scikit-learn.

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Kanton','EVU'], drop_first=True)
print("Encoded shape:", df_encoded.shape)
df_encoded.head()

## 3. Regression Example
We’ll predict the **numeric** target `DTV` from the available features. For simplicity, we’ll consider:
- `DWV`, `DNWV`, `lon`, `lat` + one-hot columns for `Kanton` and `EVU`

### 3.1 Train/Test Split
We'll do a standard 80/20 split.

In [ ]:
target_col = 'DTV'
exclude_cols = ['Code','UIC','Bahnhof','ISB_GI','Jahr','DTV']

X_cols = [c for c in df_encoded.columns if c not in exclude_cols]
X = df_encoded[X_cols].values
y = df_encoded[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train set shape:", X_train.shape, y_train.shape)
print("Test  set shape:", X_test.shape,  y_test.shape)

### 3.2 Fit a Linear Regression

In [ ]:
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

y_pred = reg_model.predict(X_test)
print("Model trained.")

### 3.3 Evaluate Regression
- **R²** (coefficient of determination)
- **MAE** (mean absolute error)
- **RMSE** (root mean squared error)

In [ ]:
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Regression Metrics:")
print(f"  R^2   = {r2:.3f}")
print(f"  MAE   = {mae:.2f}")
print(f"  RMSE  = {rmse:.2f}")

### 3.4 Visualize Predicted vs Actual

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([0, max(y_test)], [0, max(y_test)], '--r')
plt.title("Predicted vs. Actual DTV")
plt.xlabel("Actual DTV")
plt.ylabel("Predicted DTV")
plt.xlim([0, max(y_test)*1.1])
plt.ylim([0, max(y_pred)*1.1])
plt.tight_layout()
plt.show()

## 4. Classification Example
We'll create a **binary** target for “High Traffic” vs “Low Traffic,” e.g. stations with `DTV >= 1000` are **1** (high) else **0** (low).

Then we use **Logistic Regression** (as a simple classifier) and evaluate with:
- Accuracy
- Precision
- Recall
- F1 score
- Confusion matrix

In [ ]:
# Create binary target
df_encoded['HighTraffic'] = (df_encoded['DTV'] >= 1000).astype(int)

# We'll predict HighTraffic from the same set of features,
# but exclude DTV itself.
exclude_cols_clf = ['HighTraffic','DTV','Code','UIC','Bahnhof','ISB_GI','Jahr']
X_cols_clf = [c for c in df_encoded.columns if c not in exclude_cols_clf]
X_clf = df_encoded[X_cols_clf].values
y_clf = df_encoded['HighTraffic'].values

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)

clf_model = LogisticRegression(max_iter=1000)
clf_model.fit(X_train_clf, y_train_clf)

# Predict on test set
y_pred_clf = clf_model.predict(X_test_clf)

print("Classification model trained.")

### 4.1 Evaluate Classification
- **Accuracy**: overall fraction of correct predictions
- **Precision**: fraction of predicted positives that are correct
- **Recall**: fraction of actual positives that are identified
- **F1**: harmonic mean of precision and recall

Finally, we plot the **confusion matrix** as a heatmap.

In [ ]:
acc = accuracy_score(y_test_clf, y_pred_clf)
prec = precision_score(y_test_clf, y_pred_clf, zero_division=0)
rec = recall_score(y_test_clf, y_pred_clf, zero_division=0)
f1 = f1_score(y_test_clf, y_pred_clf, zero_division=0)

print("Classification Metrics:")
print(f"  Accuracy  = {acc:.3f}")
print(f"  Precision = {prec:.3f}")
print(f"  Recall    = {rec:.3f}")
print(f"  F1 Score  = {f1:.3f}")

cm = confusion_matrix(y_test_clf, y_pred_clf)
print("\nConfusion Matrix:")
print(cm)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix (HighTraffic vs. LowTraffic)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

## 5. Summary & Next Steps

1. **Regression**
   - We used **LinearRegression** to predict `DTV`.
   - Evaluated with **R²**, **MAE**, **RMSE**, and a scatter plot.

2. **Classification**
   - We defined a binary label `HighTraffic`.
   - Used **LogisticRegression** to classify stations.
   - Evaluated with **accuracy**, **precision**, **recall**, **F1**, and a confusion matrix.

### Possible Enhancements
- **Cross-validation**: More robust performance estimates.
- **Hyperparameter tuning**: `GridSearchCV` or `RandomizedSearchCV`.
- **Alternative models**: Random Forest, Gradient Boosting, etc.
- **Feature engineering**: More meaningful transformations of `DWV`, `DNWV`, location-based features, etc.
- **Handling skew**: Log-transform `DTV` or set thresholds carefully.

This completes our **model evaluation** demo!